# Apache Iceberg for Telecom Enterprises
## A Complete Guide with PySpark & Telecom Time Series Data

This notebook demonstrates how to use Apache Iceberg in telecom enterprise environments using PySpark. Apache Iceberg is an open table format for huge analytic datasets that provides:

- **ACID transactions** for reliable telecom data operations
- **Schema evolution** for adapting to new network technologies
- **Time travel** for historical network performance analysis
- **Hidden partitioning** for optimal time-series data queries
- **Data compaction** for efficient storage of large telecom datasets
- **Rollback capabilities** for data recovery scenarios

### Why Iceberg for Telecom Enterprises?
- Handle massive time-series data from network infrastructure
- Multi-engine compatibility (Spark, Flink, Trino, etc.) for different analytics needs
- Better performance for network monitoring and analytics
- Enterprise-grade data governance for regulatory compliance
- Support for real-time and batch processing of telecom metrics


## 1. Environment Setup

First, let's set up the required dependencies. This notebook handles both Google Colab and local environments automatically and will generate synthetic telecom time-series data for demonstration.


In [ ]:
import os
import platform
import subprocess
import sys

# Detect environment
system = platform.system()
in_colab = 'google.colab' in sys.modules
print(f"🖥️ OS: {system}")
print(f"📔 Environment: {'Google Colab' if in_colab else 'Local Jupyter'}")

# Install Java (required for PySpark)
if system == "Linux" and not in_colab:
    # Linux environment (not Colab)
    print("📦 Installing Java on Linux...")
    !apt-get update -qq
    !apt-get install -y openjdk-8-jdk-headless -qq
    os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
elif system == "Darwin":
    # macOS environment
    print("📦 Setting up Java on macOS...")
    try:
        java_version = subprocess.run(['java', '-version'], capture_output=True, text=True, stderr=subprocess.STDOUT)
        if java_version.returncode == 0:
            print("☕ Java is already installed")
            # Try to find JAVA_HOME
            try:
                java_home = subprocess.run(['/usr/libexec/java_home'], capture_output=True, text=True)
                if java_home.returncode == 0:
                    os.environ["JAVA_HOME"] = java_home.stdout.strip()
                    print(f"🏠 JAVA_HOME: {os.environ.get('JAVA_HOME')}")
            except:
                print("⚠️ Could not automatically set JAVA_HOME")
        else:
            print("⚠️ Java not found. Please install Java 8+ using:")
            print("   brew install openjdk@8")
    except FileNotFoundError:
        print("⚠️ Java not found. Please install Java 8+ using:")
        print("   brew install openjdk@8")
elif in_colab:
    # Google Colab - Java is pre-installed
    print("☕ Using pre-installed Java in Colab")
    # Try to set JAVA_HOME for Colab
    possible_java_homes = [
        "/usr/lib/jvm/java-11-openjdk-amd64",
        "/usr/lib/jvm/java-8-openjdk-amd64",
        "/usr/lib/jvm/default-java"
    ]
    for java_home in possible_java_homes:
        if os.path.exists(java_home):
            os.environ["JAVA_HOME"] = java_home
            print(f"🏠 JAVA_HOME set to: {java_home}")
            break

print(f"☕ JAVA_HOME: {os.environ.get('JAVA_HOME', 'Not set')}")
print("✅ Java setup completed!")


In [ ]:
# Install Python packages with dependency management
print("📦 Installing Python packages...")

# For Google Colab, use compatible versions
if in_colab:
    print("🔧 Installing Colab-compatible versions...")
    # Use existing pandas and numpy versions in Colab to avoid conflicts
    %pip install -q pyspark>=3.4.0
    %pip install -q pyiceberg --no-deps  # Install without dependencies to avoid conflicts
    %pip install -q s3fs  # For S3 support
else:
    # For local environments, install specific versions
    print("🔧 Installing packages for local environment...")
    %pip install -q pyspark==3.4.1
    %pip install -q pyiceberg[s3fs]==0.5.1
    %pip install -q pandas>=2.0.0
    %pip install -q matplotlib seaborn

print("✅ Package installation completed!")

# Import required libraries
try:
    import pandas as pd
    print(f"📊 Pandas version: {pd.__version__}")
except ImportError as e:
    print(f"⚠️ Pandas import failed: {e}")
    print("💡 Try restarting the runtime and running cells again")

try:
    import pyspark
    print(f"⚡ PySpark version: {pyspark.__version__}")
except ImportError as e:
    print(f"⚠️ PySpark import failed: {e}")
    
print("🚀 Ready to proceed!")


## 2. Spark Configuration with Iceberg

Configure Spark to work with Apache Iceberg. In enterprise environments, you would typically configure this in your cluster settings.


In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql.functions import *
import os

# Download Iceberg JAR for Spark
jar_url = "https://repo1.maven.org/maven2/org/apache/iceberg/iceberg-spark-runtime-3.4_2.12/1.4.2/iceberg-spark-runtime-3.4_2.12-1.4.2.jar"

if in_colab:
    jar_path = "/content/iceberg-spark-runtime.jar"
    warehouse_path = "/content/iceberg-warehouse"
else:
    jar_path = "./iceberg-spark-runtime.jar"
    warehouse_path = "./iceberg-warehouse"

print(f"📥 Downloading Iceberg JAR to {jar_path}...")
!wget -q {jar_url} -O {jar_path}

# Configure Spark with Iceberg
print("⚡ Initializing Spark with Iceberg...")
spark = SparkSession.builder \
    .appName("Iceberg Enterprise Demo") \
    .config("spark.jars", jar_path) \
    .config("spark.sql.extensions", "org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.iceberg.spark.SparkSessionCatalog") \
    .config("spark.sql.catalog.spark_catalog.type", "hive") \
    .config("spark.sql.catalog.local", "org.apache.iceberg.spark.SparkCatalog") \
    .config("spark.sql.catalog.local.type", "hadoop") \
    .config("spark.sql.catalog.local.warehouse", warehouse_path) \
    .config("spark.sql.warehouse.dir", warehouse_path) \
    .config("spark.serializer", "org.apache.spark.serializer.KryoSerializer") \
    .config("spark.sql.adaptive.enabled", "true") \
    .config("spark.sql.adaptive.coalescePartitions.enabled", "true") \
    .getOrCreate()

# Set log level to reduce noise
spark.sparkContext.setLogLevel("WARN")

print(f"✅ Spark {spark.version} with Iceberg initialized successfully!")
print(f"📁 Warehouse location: {warehouse_path}")
print(f"☕ Java Home: {os.environ.get('JAVA_HOME', 'Not set')}")


## 3. Generating Synthetic Telecom Time Series Data

Let's generate realistic telecom network performance data that represents typical enterprise telecom monitoring scenarios.


In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import random
import os

# Save Python's built-in min/max functions before PySpark import overwrites them
python_min = min
python_max = max

print("📡 Generating Synthetic Telecom Time Series Data...")

# Generation settings for demo (scaled down from 10GB)
chunk_size_seconds = 60  # 1 minute chunks for demo
site_count = 100  # number of unique cell sites (reduced for demo)
start_time = datetime(2023, 1, 1)
demo_chunks = 50  # Generate 50 chunks for demo (~5MB total)

# Telecom network configuration
regions = ["North", "South", "East", "West"]
cities = {
    "North": ["Dendam", "Rondburg"],
    "South": ["Schieveste"],
    "East": ["Schipstad", "Dort"],
    "West": ["Damstad"],
}
technologies = ["6G", "7G"]
vendors = ["Ericia", "Noson", "Weihu"]

# Generate site metadata
print("🏗️ Creating telecom site metadata...")
sites = []
for i in range(site_count):
    region = random.choice(regions)
    city = random.choice(cities[region])
    tech = random.choices(technologies, weights=[0.5, 0.5])[0]
    vendor = random.choices(vendors, weights=[0.4, 0.4, 0.2])[0]  # Weihu less common
    site_id = f"SITE_{i:05d}"
    sites.append({
        "site_id": site_id,
        "region": region,
        "city": city,
        "technology": tech,
        "vendor": vendor,
    })

print(f"📍 Created {len(sites)} telecom sites across {len(regions)} regions")

# Generate time series data for each site
def generate_telecom_data_chunk(second_offset):
    timestamp = start_time + timedelta(seconds=second_offset)
    records = []
    for site in sites:
        region = site["region"]
        city = site["city"]
        tech = site["technology"]
        vendor = site["vendor"]

        # Signal strength (RSSI) - better for 7G and Noson vendor
        base_rssi = -75 if tech == "7G" else -85
        if vendor == "Noson":
            base_rssi += 2
        if vendor == "Weihu":
            base_rssi -= 3
        rssi = np.random.normal(loc=base_rssi, scale=3)

        # Latency - varies by region and technology
        if region == "West" or city in ["Damstad"]:
            base_latency = 80
        else:
            base_latency = 35 if tech == "7G" else 60
        latency = np.random.normal(loc=base_latency, scale=8)

        # Data volume (in MB)
        data_volume = np.random.exponential(scale=6)

        # CPU usage - varies by technology and vendor
        base_cpu = 48 + (7 if tech == "7G" else 0) + (5 if vendor == "Weihu" else 0)
        cpu_usage = np.clip(np.random.normal(loc=base_cpu, scale=9), 0, 100)

        # Drop rate - influenced by CPU and poor-performing cities
        city_penalty = 0.04 if city in ["Damstad", "Schieveste"] else 0.01
        drop_rate = (
            python_min(1.0, 0.0012 * cpu_usage + city_penalty + np.random.beta(1, 180)) * 100
        )

        records.append({
            "timestamp": timestamp,
            "region": region,
            "city": city,
            "site_id": site["site_id"],
            "technology": tech,
            "vendor": vendor,
            "rssi_dbm": round(rssi, 2),
            "latency_ms": round(latency, 2),
            "data_volume_mb": round(data_volume, 2),
            "drop_rate_percent": round(drop_rate, 2),
            "cpu_usage_percent": round(cpu_usage, 2),
        })
    return records

# Generate telecom time series data
print("⚡ Generating telecom time series metrics...")
all_records = []
for chunk_idx in range(demo_chunks):
    chunk_records = []
    for second in range(chunk_size_seconds):
        minute_offset = chunk_idx * chunk_size_seconds + second
        chunk_records.extend(generate_telecom_data_chunk(minute_offset))
    all_records.extend(chunk_records)

# Convert to Spark DataFrame
telecom_metrics_df = spark.createDataFrame(all_records)

# Generate site metadata DataFrame
sites_df = spark.createDataFrame(sites)

print("✅ Telecom data generated successfully!")
print(f"   📡 Sites: {sites_df.count():,} unique telecom sites")
print(f"   📊 Metrics: {telecom_metrics_df.count():,} time series records")
print(f"   🕒 Time Range: {start_time} to {start_time + timedelta(seconds=demo_chunks * chunk_size_seconds)}")

# Show sample data
print("\n📍 Sample Site Metadata:")
sites_df.show(10)

print("\n📊 Sample Network Metrics:")
telecom_metrics_df.show(10)


## 4. Creating Iceberg Tables for Telecom Data

Now let's create Iceberg tables optimized for telecom time-series data with enterprise-grade configurations including time-based partitioning for optimal query performance.


In [ ]:
# Create Iceberg tables for telecom data
print("🏗️ Creating Iceberg tables for telecom data...")

# Create telecom sites table (metadata/reference data)
sites_df.write \
    .format("iceberg") \
    .mode("overwrite") \
    .saveAsTable("local.db.telecom_sites")

# Create telecom metrics table with timestamp partitioning (critical for time-series data)
telecom_metrics_df.write \
    .format("iceberg") \
    .mode("overwrite") \
    .partitionBy("timestamp") \
    .saveAsTable("local.db.telecom_metrics")

print("✅ Telecom Iceberg tables created successfully!")

# Verify tables
print("\n📋 Available Telecom Tables:")
spark.sql("SHOW TABLES IN local.db").show()

# Show table schemas
print("\n🏛️ Table Schemas:")
print("\n📡 Telecom Sites Schema:")
spark.sql("DESCRIBE local.db.telecom_sites").show()

print("\n📊 Telecom Metrics Schema:")
spark.sql("DESCRIBE local.db.telecom_metrics").show()

# Basic table statistics
print("\n📊 Telecom Table Statistics:")
sites_count = spark.sql("SELECT COUNT(*) as count FROM local.db.telecom_sites").collect()[0]['count']
metrics_count = spark.sql("SELECT COUNT(*) as count FROM local.db.telecom_metrics").collect()[0]['count']

# Calculate some basic telecom KPIs
avg_rssi = spark.sql("SELECT AVG(rssi_dbm) as avg_rssi FROM local.db.telecom_metrics").collect()[0]['avg_rssi']
avg_latency = spark.sql("SELECT AVG(latency_ms) as avg_latency FROM local.db.telecom_metrics").collect()[0]['avg_latency']
avg_drop_rate = spark.sql("SELECT AVG(drop_rate_percent) as avg_drop_rate FROM local.db.telecom_metrics").collect()[0]['avg_drop_rate']

print(f"   📡 Total Sites: {sites_count:,}")
print(f"   📊 Metrics Records: {metrics_count:,}")
print(f"   📶 Average RSSI: {avg_rssi:.2f} dBm")
print(f"   ⏱️ Average Latency: {avg_latency:.2f} ms")
print(f"   ⚡ Average Drop Rate: {avg_drop_rate:.2f}%")

# Show partition information for time-series table
print("\n🗂️ Telecom Metrics Partitions:")
spark.sql("SELECT * FROM local.db.telecom_metrics.partitions LIMIT 5").show(truncate=False)


## 5. Time Travel - Telecom Network History Analysis

Demonstrate Iceberg's powerful time travel capabilities for telecom network historical analysis, troubleshooting, and audit purposes.


In [ ]:
# Telecom Time Travel Demonstration
print("🕐 TELECOM NETWORK TIME TRAVEL DEMONSTRATION")
print("=" * 50)

# View table history
print("\n📜 Current Telecom Metrics Table History:")
spark.sql("SELECT * FROM local.db.telecom_metrics.history").show(truncate=False)

# Simulate network events - add more telecom data to create new snapshots
print("\n🔄 Simulating network performance data updates...")

# Generate additional telecom metrics (simulating network changes)
additional_records = []
for chunk_idx in range(10):  # Add 10 more minutes of data
    for second in range(60):
        minute_offset = demo_chunks * chunk_size_seconds + chunk_idx * 60 + second
        additional_records.extend(generate_telecom_data_chunk(minute_offset))

additional_metrics_df = spark.createDataFrame(additional_records)

# Append new telecom data
additional_metrics_df.write \
    .format("iceberg") \
    .mode("append") \
    .saveAsTable("local.db.telecom_metrics")

print(f"✅ Added {additional_metrics_df.count()} new telecom metrics records")

# Simulate network configuration update
print("\n📡 Simulating network configuration update...")
spark.sql("""
    UPDATE local.db.telecom_sites 
    SET technology = '7G' 
    WHERE technology = '6G' AND vendor = 'Noson' AND region = 'North'
""")

# Show updated history
print("\n📜 Updated Telecom Table History:")
spark.sql("SELECT snapshot_id, committed_at, operation FROM local.db.telecom_metrics.history ORDER BY committed_at").show(truncate=False)

# Demonstrate time travel query for network analysis
snapshots = spark.sql("SELECT snapshot_id FROM local.db.telecom_metrics.snapshots ORDER BY committed_at").collect()
if len(snapshots) >= 2:
    first_snapshot = snapshots[0]['snapshot_id']
    print(f"\n🔍 Analyzing network performance from historical snapshot: {first_snapshot}")
    
    # Query historical network data
    historical_data = spark.read \
        .format("iceberg") \
        .option("snapshot-id", first_snapshot) \
        .table("local.db.telecom_metrics")
    
    current_data = spark.read.format("iceberg").table("local.db.telecom_metrics")
    
    # Compare network performance metrics
    historical_avg_latency = historical_data.select(avg("latency_ms")).collect()[0][0]
    current_avg_latency = current_data.select(avg("latency_ms")).collect()[0][0]
    
    historical_avg_rssi = historical_data.select(avg("rssi_dbm")).collect()[0][0]
    current_avg_rssi = current_data.select(avg("rssi_dbm")).collect()[0][0]
    
    print(f"📊 Historical record count: {historical_data.count():,}")
    print(f"📊 Current record count: {current_data.count():,}")
    print(f"📈 New records added: {current_data.count() - historical_data.count():,}")
    print(f"📶 Historical avg RSSI: {historical_avg_rssi:.2f} dBm")
    print(f"📶 Current avg RSSI: {current_avg_rssi:.2f} dBm")
    print(f"⏱️ Historical avg latency: {historical_avg_latency:.2f} ms")
    print(f"⏱️ Current avg latency: {current_avg_latency:.2f} ms")
    
    # Show network performance trend
    print("\n📈 Network Performance Comparison:")
    rssi_change = current_avg_rssi - historical_avg_rssi
    latency_change = current_avg_latency - historical_avg_latency
    print(f"   RSSI Change: {rssi_change:+.2f} dBm ({'📈 Better' if rssi_change > 0 else '📉 Worse'})")
    print(f"   Latency Change: {latency_change:+.2f} ms ({'📈 Better' if latency_change < 0 else '📉 Worse'})")
    
print("\n✅ Telecom time travel demonstration completed!")


## 6. Telecom Network Analytics & Best Practices

Demonstrate real-world telecom network analytics scenarios and summarize enterprise best practices for telecom data lakes.


In [ ]:
# Telecom Network Analytics Examples
print("📡 TELECOM NETWORK ANALYTICS EXAMPLES")
print("=" * 50)

# Network Performance by Region and Technology
print("\n🌍 Network Performance by Region & Technology:")
regional_analysis = spark.sql("""
    SELECT 
        s.region,
        s.technology,
        COUNT(DISTINCT s.site_id) as site_count,
        ROUND(AVG(m.rssi_dbm), 2) as avg_rssi,
        ROUND(AVG(m.latency_ms), 2) as avg_latency,
        ROUND(AVG(m.drop_rate_percent), 2) as avg_drop_rate,
        ROUND(AVG(m.cpu_usage_percent), 2) as avg_cpu_usage
    FROM local.db.telecom_sites s
    JOIN local.db.telecom_metrics m ON s.site_id = m.site_id
    GROUP BY s.region, s.technology
    ORDER BY s.region, s.technology
""")
regional_analysis.show()

# Vendor Performance Comparison
print("\n🏭 Vendor Performance Comparison:")
vendor_analysis = spark.sql("""
    SELECT 
        s.vendor,
        s.technology,
        COUNT(DISTINCT s.site_id) as site_count,
        ROUND(AVG(m.rssi_dbm), 2) as avg_rssi,
        ROUND(AVG(m.latency_ms), 2) as avg_latency,
        ROUND(AVG(m.drop_rate_percent), 2) as avg_drop_rate
    FROM local.db.telecom_sites s
    JOIN local.db.telecom_metrics m ON s.site_id = m.site_id
    GROUP BY s.vendor, s.technology
    ORDER BY avg_drop_rate ASC
""")
vendor_analysis.show()

# Time-based Network Performance Trends
print("\n📈 Hourly Network Performance Trends:")
hourly_trends = spark.sql("""
    SELECT 
        HOUR(m.timestamp) as hour_of_day,
        COUNT(*) as measurement_count,
        ROUND(AVG(m.rssi_dbm), 2) as avg_rssi,
        ROUND(AVG(m.latency_ms), 2) as avg_latency,
        ROUND(AVG(m.data_volume_mb), 2) as avg_data_volume,
        ROUND(AVG(m.drop_rate_percent), 2) as avg_drop_rate
    FROM local.db.telecom_metrics m
    GROUP BY HOUR(m.timestamp)
    ORDER BY hour_of_day
""")
hourly_trends.show()

# Network Anomaly Detection (High Drop Rate Sites)
print("\n⚠️ Network Anomaly Detection (High Drop Rate Sites):")
anomaly_detection = spark.sql("""
    SELECT 
        s.site_id,
        s.region,
        s.city,
        s.vendor,
        s.technology,
        ROUND(AVG(m.drop_rate_percent), 2) as avg_drop_rate,
        ROUND(AVG(m.rssi_dbm), 2) as avg_rssi,
        ROUND(AVG(m.latency_ms), 2) as avg_latency
    FROM local.db.telecom_sites s
    JOIN local.db.telecom_metrics m ON s.site_id = m.site_id
    GROUP BY s.site_id, s.region, s.city, s.vendor, s.technology
    HAVING AVG(m.drop_rate_percent) > 2.0
    ORDER BY avg_drop_rate DESC
    LIMIT 10
""")
anomaly_detection.show()

# Telecom Enterprise Best Practices
print("\n📡 TELECOM ENTERPRISE BEST PRACTICES")
print("=" * 50)

telecom_practices = [
    "🕐 Time-Series Partitioning:",
    "   • Partition by timestamp (hourly/daily) for optimal query performance",
    "   • Use hidden partitioning for automatic time-based partitioning",
    "   • Consider site_id partitioning for geographically distributed queries",
    "",
    "📊 Network Monitoring:",
    "   • Implement real-time streaming with batch processing",
    "   • Set up automated anomaly detection on key KPIs",
    "   • Use time travel for network incident analysis",
    "",
    "🔧 Data Lifecycle Management:",
    "   • Implement tiered storage (hot/warm/cold) based on data age",
    "   • Regular compaction for time-series data efficiency",
    "   • Automated cleanup of old snapshots based on retention policies",
    "",
    "🔒 Regulatory Compliance:",
    "   • Maintain audit trails for network performance data",
    "   • Implement data lineage for regulatory reporting",
    "   • Use schema evolution for changing network standards",
    "",
    "⚡ Performance Optimization:",
    "   • Use columnar format for analytical queries",
    "   • Implement predicate pushdown for time-range queries",
    "   • Optimize file sizes for network metric ingestion patterns",
    "",
    "🛡️ Network Operations:",
    "   • Backup critical network configuration metadata",
    "   • Test disaster recovery for network monitoring systems",
    "   • Use Iceberg ACID properties for consistent network reporting"
]

for practice in telecom_practices:
    print(practice)

# Telecom Analytics Summary
print(f"\n\n📊 TELECOM DEMO SUMMARY")
print("=" * 40)
final_telecom_stats = spark.sql("""
    SELECT 
        'Total Network Sites' as metric,
        CAST(COUNT(*) AS STRING) as value
    FROM local.db.telecom_sites
    UNION ALL
    SELECT 
        'Total Metric Records',
        CAST(COUNT(*) AS STRING)
    FROM local.db.telecom_metrics
    UNION ALL
    SELECT 
        'Average Network RSSI',
        CONCAT(CAST(ROUND(AVG(rssi_dbm), 2) AS STRING), ' dBm')
    FROM local.db.telecom_metrics
    UNION ALL
    SELECT 
        'Average Latency',
        CONCAT(CAST(ROUND(AVG(latency_ms), 2) AS STRING), ' ms')
    FROM local.db.telecom_metrics
    UNION ALL
    SELECT 
        'Average Drop Rate',
        CONCAT(CAST(ROUND(AVG(drop_rate_percent), 2) AS STRING), '%')
    FROM local.db.telecom_metrics
""")
final_telecom_stats.show(truncate=False)

print("\n✅ Apache Iceberg Telecom Enterprise Demo Completed!")
print("📡 Ready for production telecom data lake deployment!")
print("📚 Learn more: https://iceberg.apache.org/")

# Cleanup
print("\n🧹 Cleaning up...")
spark.stop()
print("✅ Spark session closed.")


# Apache Iceberg for On-Premise Enterprises
## A Complete Guide with PySpark

This notebook demonstrates how to use Apache Iceberg in on-premise enterprise environments using PySpark. Apache Iceberg is an open table format for huge analytic datasets that provides:

- **ACID transactions**
- **Schema evolution**
- **Time travel**
- **Hidden partitioning**
- **Data compaction**
- **Rollback capabilities**

### Why Iceberg for Enterprises?
- Reliable data lake operations
- Multi-engine compatibility (Spark, Flink, Trino, etc.)
- Better performance through advanced optimization
- Enterprise-grade data governance


## 1. Environment Setup

First, let's install the required dependencies for Google Colab environment.


In [2]:
# Install Java 8 (required for PySpark)
!apt-get update
!apt-get install -y openjdk-8-jdk-headless
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

# Install PySpark with Iceberg support
%pip install pyspark==3.4.1
%pip install pyiceberg[s3fs,duckdb]==0.5.1
%pip install pandas==2.0.3
%pip install matplotlib seaborn

print("✅ Dependencies installed successfully!")


zsh:1: command not found: apt-get
zsh:1: command not found: apt-get
Note: you may need to restart the kernel to use updated packages.
zsh:1: no matches found: pyiceberg[s3fs,duckdb]==0.5.1
Note: you may need to restart the kernel to use updated packages.
  Using cached pandas-2.0.3.tar.gz (5.3 MB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for pandas: filename=pandas-2.0.3-cp312-cp312-macosx_11_0_arm64.whl size=10329889 sha256=ee7f217bea24162b3e3209723c5e401e2d0da8cdd062c8524bcb1804f75c557c
  Stored in directory: /Users/codinggents/Library/Caches/pip/wheels/08/95/b7/15a2a9958c1fde0807c23b05bfed1a32ff9c7225c55d270d27
Successfully built pandas
  Attempting uninstall: pandas
    Found existing installation: pandas 2.2.2
    Uninstalling pandas-2.2.2:
      Successfully uninstalled pandas-2.2.2
ERROR: pip's dependency resolver does not currently take into account all the packages tha

## 2. Spark Configuration with Iceberg

Configure Spark to work with Apache Iceberg. In enterprise environments, you would typically configure this in your cluster settings.


In [3]:
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql.functions import *
import pandas as pd
import os

# Download Iceberg JAR for Spark
!wget -q https://repo1.maven.org/maven2/org/apache/iceberg/iceberg-spark-runtime-3.4_2.12/1.4.2/iceberg-spark-runtime-3.4_2.12-1.4.2.jar -O /content/iceberg-spark-runtime.jar

# Configure Spark with Iceberg
spark = SparkSession.builder \
    .appName("Iceberg Enterprise Demo") \
    .config("spark.jars", "/content/iceberg-spark-runtime.jar") \
    .config("spark.sql.extensions", "org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.iceberg.spark.SparkSessionCatalog") \
    .config("spark.sql.catalog.spark_catalog.type", "hive") \
    .config("spark.sql.catalog.local", "org.apache.iceberg.spark.SparkCatalog") \
    .config("spark.sql.catalog.local.type", "hadoop") \
    .config("spark.sql.catalog.local.warehouse", "/content/iceberg-warehouse") \
    .config("spark.sql.warehouse.dir", "/content/iceberg-warehouse") \
    .getOrCreate()

# Set log level to reduce noise
spark.sparkContext.setLogLevel("WARN")

print(f"✅ Spark {spark.version} with Iceberg initialized successfully!")
print(f"📁 Warehouse location: /content/iceberg-warehouse")


/content/iceberg-spark-runtime.jar: No such file or directory


/opt/anaconda3/lib/python3.12/site-packages/pyspark/bin/spark-class: line 71: /usr/lib/jvm/java-8-openjdk-amd64/bin/java: No such file or directory
/opt/anaconda3/lib/python3.12/site-packages/pyspark/bin/spark-class: line 97: CMD: bad array subscript
head: illegal line count -- -1


RuntimeError: Java gateway process exited before sending its port number

## 3. Creating Sample Enterprise Data

Let's create sample datasets that represent typical enterprise scenarios:
- Customer data
- Sales transactions
- Product catalog


In [ ]:
from datetime import datetime, timedelta
import random

# Generate sample customer data
def generate_customer_data(num_customers=1000):
    customers = []
    for i in range(num_customers):
        customers.append({
            'customer_id': f'CUST_{i:06d}',
            'first_name': f'FirstName{i}',
            'last_name': f'LastName{i}',
            'email': f'customer{i}@enterprise.com',
            'registration_date': datetime(2020, 1, 1) + timedelta(days=random.randint(0, 1400)),
            'customer_segment': random.choice(['Premium', 'Standard', 'Basic']),
            'credit_limit': random.randint(1000, 50000),
            'country': random.choice(['USA', 'Canada', 'UK', 'Germany', 'France']),
            'is_active': random.choice([True, False])
        })
    return customers

# Generate sample sales data
def generate_sales_data(num_transactions=5000):
    sales = []
    for i in range(num_transactions):
        sales.append({
            'transaction_id': f'TXN_{i:08d}',
            'customer_id': f'CUST_{random.randint(0, 999):06d}',
            'product_id': f'PROD_{random.randint(1, 100):03d}',
            'transaction_date': datetime(2023, 1, 1) + timedelta(days=random.randint(0, 365)),
            'quantity': random.randint(1, 10),
            'unit_price': round(random.uniform(10.0, 500.0), 2),
            'discount_percentage': random.uniform(0, 0.3),
            'payment_method': random.choice(['Credit Card', 'Debit Card', 'Cash', 'Bank Transfer']),
            'sales_rep': f'REP_{random.randint(1, 50):03d}'
        })
    return sales

# Create DataFrames
customers_data = generate_customer_data(1000)
sales_data = generate_sales_data(5000)

customers_df = spark.createDataFrame(customers_data)
sales_df = spark.createDataFrame(sales_data)

# Add calculated columns
sales_df = sales_df.withColumn(
    'total_amount', 
    col('quantity') * col('unit_price') * (1 - col('discount_percentage'))
)

print("✅ Sample data generated:")
print(f"   📊 Customers: {customers_df.count():,} records")
print(f"   💰 Sales: {sales_df.count():,} transactions")
print(f"   💵 Total revenue: ${sales_df.select(sum('total_amount')).collect()[0][0]:,.2f}")


## 4. Creating Iceberg Tables

Now let's create Iceberg tables. In enterprise environments, these would typically be stored in distributed storage systems like HDFS, S3, or Azure Data Lake.


In [ ]:
# Create customers Iceberg table
customers_df.write \
    .format("iceberg") \
    .mode("overwrite") \
    .option("path", "/content/iceberg-warehouse/customers") \
    .saveAsTable("local.db.customers")

# Create sales Iceberg table with partitioning (enterprise best practice)
sales_df.write \
    .format("iceberg") \
    .mode("overwrite") \
    .option("path", "/content/iceberg-warehouse/sales") \
    .partitionBy("transaction_date") \
    .saveAsTable("local.db.sales")

print("✅ Iceberg tables created successfully!")

# Show table information
print("\n📋 Table Details:")
spark.sql("SHOW TABLES IN local.db").show()

# Show customers table schema
print("\n👥 Customers Table Schema:")
spark.sql("DESCRIBE local.db.customers").show()

# Show sales table schema
print("\n💰 Sales Table Schema:")
spark.sql("DESCRIBE local.db.sales").show()


## 5. Basic Iceberg Operations

Let's explore basic operations that are crucial for enterprise data management.


In [ ]:
# Read Iceberg tables
customers_iceberg = spark.read.format("iceberg").table("local.db.customers")
sales_iceberg = spark.read.format("iceberg").table("local.db.sales")

print("📊 Data Summary:")
print(f"   Customers: {customers_iceberg.count():,}")
print(f"   Sales Transactions: {sales_iceberg.count():,}")

# Sample queries
print("\n🔍 Top 5 Customer Segments by Count:")
customers_iceberg.groupBy("customer_segment") \
    .count() \
    .orderBy(col("count").desc()) \
    .show()

print("\n💳 Sales by Payment Method:")
sales_iceberg.groupBy("payment_method") \
    .agg(
        count("*").alias("transaction_count"),
        round(sum("total_amount"), 2).alias("total_revenue")
    ) \
    .orderBy(col("total_revenue").desc()) \
    .show()


## 6. Time Travel - Enterprise Data Recovery

One of Iceberg's most powerful features for enterprises is time travel, allowing you to query data as it existed at any point in time.


In [ ]:
# View table history
print("📜 Sales Table History:")
spark.sql("SELECT * FROM local.db.sales.history").show(truncate=False)

# Get current snapshot info
print("\n📸 Current Snapshots:")
spark.sql("SELECT * FROM local.db.sales.snapshots").show(truncate=False)

# Make some changes to demonstrate time travel
print("\n🔄 Making changes to demonstrate time travel...")

# Add new sales data (simulating new transactions)
new_sales_data = generate_sales_data(100)
new_sales_df = spark.createDataFrame(new_sales_data)
new_sales_df = new_sales_df.withColumn(
    'total_amount', 
    col('quantity') * col('unit_price') * (1 - col('discount_percentage'))
)

# Append to existing table
new_sales_df.write \
    .format("iceberg") \
    .mode("append") \
    .saveAsTable("local.db.sales")

print(f"✅ Added {new_sales_df.count()} new transactions")
print(f"📊 Total transactions now: {spark.read.format('iceberg').table('local.db.sales').count():,}")

# Show updated history
print("\n📜 Updated Table History:")
spark.sql("SELECT * FROM local.db.sales.history ORDER BY made_current_at").show(truncate=False)


## 7. Schema Evolution - Enterprise Agility

Schema evolution allows enterprises to adapt their data structures without breaking existing pipelines.


In [ ]:
# Show current schema
print("📋 Current Customers Schema:")
spark.sql("DESCRIBE local.db.customers").show()

# Add a new column (common enterprise requirement)
print("\n🔧 Adding new column: loyalty_points")
spark.sql("""
    ALTER TABLE local.db.customers 
    ADD COLUMN loyalty_points INT AFTER credit_limit
""")

# Show updated schema
print("\n📋 Updated Schema:")
spark.sql("DESCRIBE local.db.customers").show()

# Update some records with loyalty points
print("\n📝 Updating loyalty points for Premium customers...")
spark.sql("""
    UPDATE local.db.customers 
    SET loyalty_points = CAST(credit_limit * 0.1 AS INT)
    WHERE customer_segment = 'Premium'
""")

# Verify the update
print("\n✅ Premium customers with loyalty points:")
spark.sql("""
    SELECT customer_segment, 
           COUNT(*) as customer_count,
           AVG(loyalty_points) as avg_loyalty_points
    FROM local.db.customers 
    WHERE customer_segment = 'Premium'
    GROUP BY customer_segment
""").show()

# Show that old queries still work (backward compatibility)
print("\n🔄 Backward compatibility check - old queries still work:")
spark.sql("""
    SELECT customer_segment, COUNT(*) as count
    FROM local.db.customers 
    GROUP BY customer_segment
    ORDER BY count DESC
""").show()


## 8. Enterprise Best Practices Summary

Key recommendations for using Apache Iceberg in enterprise environments.


In [ ]:
# Display best practices and final summary
print("🏢 ENTERPRISE APACHE ICEBERG BEST PRACTICES")
print("=" * 60)

best_practices = [
    "🎯 **Partitioning Strategy**",
    "   • Use date/time partitioning for time-series data",
    "   • Consider business-specific partitions (region, department)",
    "   • Avoid over-partitioning (aim for 100MB+ per partition)",
    "",
    "🔧 **Table Maintenance**",
    "   • Schedule regular compaction jobs",
    "   • Implement snapshot cleanup policies",
    "   • Monitor table statistics and file counts",
    "",
    "🔒 **Data Governance**",
    "   • Use schema evolution carefully with proper testing",
    "   • Implement data lineage tracking",
    "   • Set up proper access controls and auditing",
    "",
    "📊 **Performance Optimization**",
    "   • Use vectorized readers when available",
    "   • Implement predicate pushdown in queries",
    "   • Optimize file sizes (128MB-1GB per file)",
    "",
    "🛡️ **Reliability & Recovery**",
    "   • Implement backup strategies for metadata",
    "   • Test disaster recovery procedures",
    "   • Use time travel for audit and compliance",
    "",
    "🔗 **Integration**",
    "   • Standardize on Iceberg across analytics engines",
    "   • Implement proper CI/CD for schema changes",
    "   • Use catalog services for metadata management"
]

for practice in best_practices:
    print(practice)

# Final summary
print("\n\n📈 DEMO SUMMARY")
print("=" * 30)

summary_stats = spark.sql("""
    SELECT 
        'Total Customers' as metric,
        CAST(COUNT(*) AS STRING) as value
    FROM local.db.customers
    UNION ALL
    SELECT 
        'Total Sales Transactions',
        CAST(COUNT(*) AS STRING)
    FROM local.db.sales
    UNION ALL
    SELECT 
        'Total Revenue',
        CONCAT('$', CAST(ROUND(SUM(total_amount), 2) AS STRING))
    FROM local.db.sales
    UNION ALL
    SELECT 
        'Active Customers',
        CAST(SUM(CASE WHEN is_active THEN 1 ELSE 0 END) AS STRING)
    FROM local.db.customers
""")

summary_stats.show(truncate=False)

print("\n✅ Apache Iceberg Enterprise Demo Completed Successfully!")
print("\n🚀 Ready for production deployment with proper configuration!")

# Optional cleanup
print("\n⏹️ Stopping Spark session...")
spark.stop()
print("✅ Demo completed! Your Iceberg tables are preserved in /content/iceberg-warehouse/")
print("📚 To learn more, visit: https://iceberg.apache.org/")
